# CREATING DATAFRAME

In [1]:
import numpy as np
import os
import conf as config
import creating_DF as cdf
import cv2
import random
import glob
import pandas as pd
from collections import Counter
from random import sample,seed
seed(123)

We need to make sure we have pictures of ourselves and from random people in all of our folders in a balanced way (positive, negative and anchor).

⚠️⚠️⚠️ `This code was ran once with some variations before the folders were correctly filled with the desired images, you only need to run the last cells`

**We create lists with all the people from LFW dataset**

In [ ]:
dir_list = os.listdir(config.NEG_PATH)
neg_list = sorted(dir_list)

**We get the names and we look for those people that appears more than 10 times**

In [ ]:
name = [person[:-9] for person in neg_list]
freq_person = Counter(name)
pairs = freq_person.items()
filtered_people = {key: value for key, value in pairs if value > 9}

In [ ]:
filtered_people 

In [ ]:
for key in filtered_people.keys():
    if key != 'Julieta_Staryfurman' and key != 'Sahar_Garber' and key != 'Noam_Pinhas_Cohen':
        filtered_people[key] = filtered_people[key] // 2

In [ ]:
#nr_rows = 3 * (25**2)
nr_rows = 0
for value in filtered_people.values():
    nr_rows += (value**2)


In [ ]:
len(filtered_people)

In [ ]:
nr_rows

In [ ]:
filtered_people

**We save half of the pictures of those people that appears more than 10 times in the anchor folder and the other half in the positive file**

In [ ]:
new_path_anchor = []
new_path_positive = []
for k,v in filtered_people.items():
    for i in range(1,v//2+1):
        new_path_anchor.append(k + '_' + '0'*(4-len(str(i))) + str(i) +'.jpg')
    for i in range(v//2+1,((v//2)*2)+1):
        new_path_positive.append(k + '_' + '0'*(4-len(str(i))) + str(i) +'.jpg')

In [ ]:
len(new_path_anchor) == len(new_path_positive) 

**We changed the name of the pictures of ourselves so that it follows the same logic as in the lfw dataset.**

In [ ]:
path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/sahar/anchor'
name = 'Sahar_Garber'

In [ ]:
cdf.changing_names(path,name)

**We make sure we have all the images in all of the folders**

In [ ]:
src_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative/'
dest_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/anchor/'

In [ ]:
cdf.add_to_folder(new_path_anchor, src_path, dest_path)

In [ ]:
pos_images = os.listdir(config.POS_PATH)
anchor_images = os.listdir(config.ANC_PATH)
all_images = pos_images + anchor_images

In [ ]:
src_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/positive/'
dest_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative/'

In [ ]:
sample_all_images = sample(list(set(all_images)),len(pos_images))

In [ ]:
cdf.add_to_folder(pos_images, src_path, dest_path)

In [ ]:
src_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative/'
dest_path = '/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative_cleaned/'

In [ ]:
add_to_folder(sample_all_images, src_path, dest_path)

🚀🚀🚀**We create the dataset. This are the only cells you should run!!!**

In [2]:
anchor, positive, negative = cdf.load_data()

(56434,)
(56434,)
(56434,)


In [3]:
df = cdf.create_df(anchor,positive,negative)

In [4]:
df = cdf.delete_bad_labels(df)

In [5]:
cdf.save_df(df)

In [6]:
#### POTENCIALLY WILL BE DELETED

In [ ]:
def rescale_resize(img):
    """
    This function rescales a given image
    :param img: image to resize
    :return: resized image
    """
    return cv2.resize(img, (105, 105)) / 255

In [ ]:
def load_from_path(path,nr_rows,filtered_people):
    """
    This function loads from a given path of a folder, all the content to an array
    :param path: the path of the folder
    :param img_num: number of images in the folder
    :return: np array with the content of the folder
    """
    imgs = []
    dir_list = os.listdir(path)
    
    if 'negative_cleaned' in path:
        for i in range(nr_rows):
            sample = random.sample(dir_list,1)[0]
            img = cv2.imread(os.path.join(path, sample))
            img = rescale_resize(img)
            imgs.append(img)
        
    
    
    else:
        dir_list = sorted(dir_list)

        img_num = len(dir_list)
        for i in range(img_num):
            if 'anchor' in path:
                img = cv2.imread(os.path.join(path, dir_list[i]))
                img = rescale_resize(img)       
                for _ in range(filtered_people[dir_list[i][:-9]]):
                    imgs.append(img)

            elif 'positive' in path:
                name = dir_list[i][:-9]
                if name != "":
                    files = []
                    for name in glob.glob(path + name + '*'):
                        files.append(name)
                    files = sorted(files)
                    for file in files:
                        if file != "":
                            img = cv2.imread(file)
                            img = rescale_resize(img)
                            imgs.append(img)


   
    return np.array(imgs)

In [ ]:
def df_with_path(path,nr_rows,filtered_people):
    """
    This function loads from a given path of a folder, all the content to an array
    :param path: the path of the folder
    :param img_num: number of images in the folder
    :return: np array with the content of the folder
    """
    paths = []
    dir_list = os.listdir(path)
    
    if 'negative_cleaned' in path:
        for i in range(nr_rows):
            sample = random.sample(dir_list,1)[0]
            paths.append(sample)
    
    else:
        dir_list = sorted(dir_list)
        
        img_num = len(dir_list)
        for i in range(img_num):
            if 'anchor' in path:       
                for _ in range(filtered_people[dir_list[i][:-9]]):
                    paths.append(dir_list[i])

            elif 'positive' in path:
                name = dir_list[i][:-9]
                if name != "":
                    files = []
                    for name in glob.glob(path + name + '*'):
                        files.append(name)
                    files = sorted(files)
                    for file in files:
                        if file != "":
                            paths.append(file.split("/")[-1])
                     


   
    return np.array(paths)

In [ ]:
pos = df_with_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/positive/',nr_rows,filtered_people)

In [ ]:
print(pos[:140])

In [ ]:
#pos.shape

In [ ]:
#pos.shape[0] - nr_rows 

In [ ]:
def load_data():
    """
    This function loads the data to three np.array (anchor, positive, negative)
    :param img_num: number of images to load (by default 2271)
    :return: a tuple with the np arrays X and y
    """
    anchor = df_with_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/anchor/',56434,filtered_people )
    print(anchor.shape)
    positive = df_with_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/positive/',56434,filtered_people )
    print(positive.shape)
    negative = df_with_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative_cleaned/',56434,filtered_people )
    print(negative.shape)
    return (anchor, positive, negative)

In [ ]:
anchor, positive, negative = load_data()

In [ ]:
anchor

In [ ]:
def create_df(anchor,positive,negative,nr_rows):
    df_1 = pd.DataFrame()
    df_1['img_1'] = anchor
    df_1['img_2'] = positive
    df_1['label'] = np.ones(nr_rows)
    
    
    df_2 = pd.DataFrame()
    df_2['img_1'] = anchor
    df_2['img_2'] = negative
    df_2['label'] = np.zeros(nr_rows)
    
    df = pd.concat([df_1,df_2])
    return df.reset_index().drop(columns=['index'])

In [ ]:
df = create_df(anchor,positive,negative,nr_rows)

In [ ]:
df.head(20)

In [ ]:
def delete_bad_labels(df):
    aux = 0
    for index,row in df.iterrows():
        if row.label == 0:
            if row.img_1[:-9] == row.img_2[:-9]:
                df.drop(index,inplace=True)
                aux += 1
    return df

In [ ]:
aux

In [ ]:
df.to_csv('dataframe.csv')

In [ ]:
df.sample(20)

In [ ]:
def load_data():
    """
    This function loads the data to three np.array (anchor, positive, negative)
    :param img_num: number of images to load (by default 2271)
    :return: a tuple with the np arrays X and y
    """
    anchor = load_from_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/anchor/',56434,filtered_people )
    print(anchor.shape)
    positive = load_from_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/positive/',56434,filtered_people )
    print(positive.shape)
    negative = load_from_path('/Users/julietastaryfurmanshalom/OneDrive/ITC/Face_Rec/data/negative_cleaned/',56434,filtered_people )
    print(negative.shape)
    return (anchor, positive, negative

In [ ]:
anchor,positive,negative = load_data()

In [ ]:
len(anchor[0:128])

In [ ]:
def create_batches(anchor, positive, negative):
    nr_batch = 0
    for i in range(0,len(anchor), 128):
        top = min(i + 128, len(anchor))
        X = np.concatenate([np.array([anchor[i:top], positive[i:top]]), np.array([anchor[i:top], negative[i:top]])], axis=1)
        y = np.concatenate([np.ones(128), np.zeros(128)], axis=0) 
        nr_batch += 1
        np.save("dataset/X_batch_" + str(nr_batch) , X)
        np.save("dataset/y_batch_" + str(nr_batch) , y)

In [ ]:
create_batches(anchor, positive, negative)